# Flexible Architecure with Optuna Optimization

In [ ]:
!pip install torchmetrics
!pip intall optuna

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

import torchmetrics
from torchmetrics import Accuracy, Precision, Recall, F1Score

import optuna

import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# Dataset & DataLoaders

In [ ]:
def get_dataloaders(batch_size):

    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))
        
    ])

    val_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))
        
    ])

    train_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=train_transform
    )
    
    val_dataset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=val_transform
    )

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    return train_loader, val_loader
